# Welcome to this tutorial on the Naive Bayes Algorithm! 

By: Spencer Karp (VP of Analytics) spk61@georgetown.edu \\
Fall 2022 \\



In this Colab we will build the Naive Bayes Algorithm from scratch with explanations along the way.

## Building the Model

Here's a tutorial on how to build the model from scratch. We will not use any tools from the sklearn library so we can understand the theory behind the model before we implement it in practice. We will only use tools from the pandas and math libraries.

### Getting the data
Let's start by importing our data. The data we will be using gives us the blood glucose levels as well as the blood pressures of almost 1000 patients. Of this group, some of the patients are diabetic and some are not. Let's see if we can build an algorithm to predict patients with diabetes by examining just their glucose levels and blood pressure.

In [58]:
# from google.colab import files
import pandas as pd
import numpy as np

# uploaded = files.upload()

In [37]:
# import data
data = pd.read_csv('Naive-Bayes-Classification-Data.csv')
print(data.head())

   glucose  bloodpressure  diabetes
0       40             85         0
1       40             92         0
2       45             63         1
3       45             80         0
4       40             73         1


Since the data is already randomized, we can just split the data in two. We'll use 30% of the data to train the model and the other 70% to test it. It should be noted that we will not be using a validation data set. A validation set can be used to prevent overfitting. The data we split now will be used to test our functions as we build the model.

In [38]:
trainingData = data[:300]
testingData = data[300:]
print(trainingData)

     glucose  bloodpressure  diabetes
0         40             85         0
1         40             92         0
2         45             63         1
3         45             80         0
4         40             73         1
..       ...            ...       ...
295       50             65         1
296       60             77         1
297       40             68         1
298       45             88         1
299       50             77         1

[300 rows x 3 columns]


### Building the training components
Let's start by building a function to organize the training data. We'll take the last column of the dataset (which holds our classifications) and build a dictionary with keys that are named with the different classifications. We'll use this organized dictionary to get our probabilities for the training set.

In [42]:
def organize_data(data):
  # Find all possible classifications
  unique = data.iloc[:,-1].unique()
  organizedData = {}

  # Build dictionary by organizing data for each classification
  for i in unique:
    temp = data.loc[data.iloc[:,-1] == i]
    organizedData[i] = temp

  return organizedData

In [43]:
# Let's test this on our training set
organizedTrainingData = organize_data(trainingData)

for key in organizedTrainingData:
  print(key)
  print(organizedTrainingData[key])

0
     glucose  bloodpressure  diabetes
0         40             85         0
1         40             92         0
3         45             80         0
5         45             82         0
6         40             85         0
..       ...            ...       ...
277       40             82         0
279       45             90         0
282       50             83         0
290       45             92         0
293       40             87         0

[152 rows x 3 columns]
1
     glucose  bloodpressure  diabetes
2         45             63         1
4         40             73         1
7         30             63         1
8         65             65         1
10        35             73         1
..       ...            ...       ...
295       50             65         1
296       60             77         1
297       40             68         1
298       45             88         1
299       50             77         1

[148 rows x 3 columns]


Now we are going to summarize our data set. For each classification, we will find the mean and standard deviation of each feature. We will also take the counts of each to aid in our probability calculations.

We'll return a dictionary with keys that are the classifications and within each key, a summary for each feature.

In [44]:
def summarize_by_class(data):

  summaries = dict()

  # Build a summary for each classification (classType)
  for classType in data:
    typeSummary=[]

    # Cycle through each feature and take summaries
    for column in data[classType]:
      summary = [data[classType][column].values.mean(), data[classType][column].values.std(), len(data[classType][column].values)]
      typeSummary.append(summary)

    # Remove summary of classification column
    del(typeSummary[-1])
    
    # Add summaries for the classification
    summaries[classType] = typeSummary

  return summaries

In [47]:
# Let's test it...
summaries = summarize_by_class(organizedTrainingData)

for key in summaries:
  print(key)
  print(summaries[key])

0
[[43.58552631578947, 3.1632014074422012, 152], [87.03947368421052, 4.697257849207244, 152]]
1
[[45.0, 9.263412481953441, 148], [71.01351351351352, 5.667577878298113, 148]]


### Building out testing components

Let's build our inverst gaussian distribution function based on the equation we have above. As a reminder, the formula looks like this:

$f(x|\mu,\sigma) = \frac{1}{\sqrt{2\pi\sigma^2}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}$

In [48]:
from math import sqrt
from math import pi
from math import exp

def calculate_probability(x, mean, stdev):
	exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
	return (1 / (sqrt(2 * pi) * stdev)) * exponent

Next, we're going to write a function that takes in a row from the testing dataset and uses Bayes's Theorem to calculate the probability that the items belong to each classification. We'll begin by creating an empty dictionary. Then for each possible classification, we'll calculate the probability that the item belongs there. We do this using the expanded form of bayes theorem that is shown above. We'll call the calculate_probability function each time and multiply these together to get each probability. 

In [49]:
def calculate_class_probabilities(summaries, row):
  # Sum up the total about of rows in training data - this will be used in the P(type) probability
  total_rows = sum([summaries[label][0][2] for label in summaries])
  probabilities = dict()

  # Cycle through each classification possible
  for classType, class_summaries in summaries.items():

    # Calculates our p(type) term
    probabilities[classType] = summaries[classType][0][2]/float(total_rows)

    # For each feature, we'll calculate p(type | feature) and multiply these to get the total probability
    for i in range(len(class_summaries)):
      mean, stdev, count = class_summaries[i]
      probabilities[classType] *= calculate_probability(row[i], mean, stdev)
      
  # We return a dictionary containing the probabilities of each classification
  return probabilities

In [50]:
# Lets test it
print(testingData.iloc[260,:])

print(calculate_class_probabilities(summaries, testingData.iloc[260,:]))

glucose          45
bloodpressure    73
diabetes          1
Name: 560, dtype: int64
{0: 5.6403433888965184e-05, 1: 0.001406421472597329}


Notice how the probability for class 1 is larger than that of class 0. Looks like our model correctly identified this row to be someone with Diabetes.

In the previous function, you'll probably have noticed that I had to examine the probabilities and tell you the model was working. This function will actually make a prediction from our given probabilities. It will first call the previous calculate_class_probabilities function and then based on that, it will identify the highest probability and slap the label on that row.

In [51]:
def predict(summaries, row):
	probabilities = calculate_class_probabilities(summaries, row)
	best_label, best_prob = None, -1
	
	# Cycle through each key of the probabilities dictionary and identify the highest one
	for class_value, probability in probabilities.items():
		if best_label is None or probability > best_prob:
			best_prob = probability
			# Pull the label too which will be our return value
			best_label = class_value
	return best_label

In [52]:
# Let's test it on that same row
print(predict(summaries, testingData.iloc[260,:]))

1


Now we can see this function actually made a prediction, and it's the right one! Pun intended ;)

### Running the Model

Now, using everything we just built, let's run and test the model on the whole data set.

Our first function will randomly split the data set into a training set and a testing set. The split will be based on the ratio we input (e.g. a .3 ratio will mean 30% of the data is used for training and the other 70% for testing).

In [53]:
def train_test_split(data, ratio):
  # Pull a random sample of the data set
  train = data.sample(frac = ratio)
  # Creating dataframe with rest of values
  test = data.drop(train.index)

  return train, test

Now, we'll combine all of the functions we built into one naive_bayes function. We'll first organize the training data, then pull summaries of the training data. Finally, from this we'll make our predictions. We'll return an array of all the predictions for the testing set!

In [54]:
def naive_bayes(trainingData, testingData):
  # Organize training data and get summaries
  organizedData = organize_data(trainingData)
  summaries = summarize_by_class(organizedData)
  
  # Build out predictions
  predictions = [];
  for i in range(len(testingData)):
    predictions.append(predict(summaries, testingData.iloc[i,:]))
  
  return predictions



Next, we'll build a function to evaluate our algorithm (there are libraries that can do this, but we'll do this from scratch). It'll do two things simultaneously. It will give us an accuracy metric and build a confusion matrix. The accuracy metric is simple. We'll calculate the percentage of the testing data that was correctly identified. \\
The confusion matrix is a bit more...confusing. The rows of the confusion matrix represent the actual classifications. The columns represent the model's classifications. The ultimate goal is to have all items on the main diagonal (0s match 0s, 1s match 1s etc.). An example can be seen a few cells below. \\
Note: If your still confused about the confusion matrix, stay tuned. We'll be releasing a conceptual article about it on the HoyAlytics Medium soon... 

In [55]:
def evaluate_algorithm(testingSet, predictions):
  # This variable will be used to calculate accuracy %
  correct = 0;

  # Use the amount of classes in the data to get the size of the confusion matrix
  unique = testingSet.iloc[:,-1].unique() 
  rows, cols = len(unique), len(unique)
  
  # Build confusion matrix with 0s
  confusionMatrix = [[0 for i in range(cols)] for j in range(rows)]

  # Loop through all predictions
  for i in range(len(predictions)):
    # Pair the prediction with the actuall classification and place in appropriate column
    confusionMatrix[testingSet.iloc[i,-1]][predictions[i]] += 1
    
    # If prediction and actual classification match, then prediction is correct
    if predictions[i] == testingSet.iloc[i,-1]:
      correct+=1
  
  return correct/len(predictions), confusionMatrix

Finally! Let's test our model. We'll first split the data (70% of our data will train the model), then send it into the model, and then evaluate it. Let's see what we get...

In [56]:
trainingSet, testingSet = train_test_split(data, 0.3)

predictions = naive_bayes(trainingSet, testingSet)
accuracy, confusion = evaluate_algorithm(testingSet, predictions)

print(accuracy)
for row in confusion:
  print(row)


0.9354375896700143
[319, 28]
[17, 333]


Looks like the model performed pretty well! Let's compare it to sklearn's naive bayes algorithm to ensure we're getting this right. We'll use the same train, test split and see what happens... 

In [57]:
# Using sklearn
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix


features = data.iloc[:,:-1]
actual = data.iloc[:,-1]

trainingFeatures, testingFeatures, trainingResults, testingResults = train_test_split(features, actual, test_size=0.7, random_state=8)

model = GaussianNB()
model.fit(trainingFeatures, trainingResults)
skLearnPredictions = model.predict(testingFeatures)


print(accuracy_score(testingResults, skLearnPredictions))
print(confusion_matrix(testingResults, skLearnPredictions))

0.9296987087517934
[[327  25]
 [ 24 321]]


These results look very similar to ours. Ultimately, they should since we built the same model. 

And that's it. We've just built a Gaussian Naive Bayes predictor! If you're going to use this model on a new set of data it's important to remember a couple of things: 

1. All your features must be continuous. That's where the "Gaussian" comes in. 
2. This model assumes that all of the features are independent (hence the word "naive"). While that's very rare in the real world, we can still gain vaulable insights if our features are only lightly dependent on each other. 

## End Notes

I want to shout out Jason Brownlee and [his article](https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/) on Naive Bayes. Also check out further reading materials on Naive Bayes in my reading list on my Medium Page. 